<a href="https://colab.research.google.com/github/markPranav/Machine-Learning/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
# try:
#   # %tensorflow_version only exists in Colab.
#   %tensorflow_version 2.9
# #   !pip install tensorflow==2.9
# except Exception:
#   pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
VOCAB_SIZE = 10000
MAXLEN = 250
BATCH_SIZE = 64

In [ ]:
train_data = pd.read_csv(train_file_path, delimiter="\t", header=None, names=["label", 'sms'])
train_data.head()

In [ ]:
test_data = pd.read_csv(test_file_path, delimiter="\t", header=None, names=["label", 'sms'])
test_data.head()

In [ ]:
train_label = train_data.pop('label')
test_label = test_data.pop('label')


In [ ]:
train_label = pd.Series(np.where(train_label.values == 'spam', 1, 0),
          train_label.index)
test_label = pd.Series(np.where(test_label.values == 'spam', 1, 0),
          test_label.index)

In [ ]:
tokenizer = Tokenizer(num_words = VOCAB_SIZE)
tokenizer.fit_on_texts(train_data['sms'])
word_index = tokenizer.word_index


In [ ]:
sequences = tokenizer.texts_to_sequences(train_data['sms'])
train_data=pad_sequences(sequences, MAXLEN)

In [ ]:
sequences = tokenizer.texts_to_sequences(test_data['sms'])
test_data=pad_sequences(sequences, MAXLEN)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.summary()

In [ ]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

In [ ]:
train_label

In [ ]:
history = model.fit(train_data, train_label, epochs=10, validation_split=0.2)

In [ ]:
results = model.evaluate(test_data, test_label)
print(results)

In [ ]:
def encode_text(text):
  sequences = tokenizer.texts_to_sequences([text])
  return pad_sequences(sequences, MAXLEN)[0]

text = "ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it."
encoded = encode_text(text)
print(encoded)

In [ ]:
reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]
  
print(decode_integers(encoded))

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    
    encoded_text = encode_text(pred_text)
    pred = np.zeros((1,250))
    pred[0] = encoded_text
    prediction = model.predict(pred) 
    label = "ham" if round(prediction[0][0]) == 0 else "spam"
    return [prediction[0][0], label]

pred_text = "please call our customer service representative"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
